The Puzzlor Payload Balance problem

In [133]:
from pulp import *
import numpy as np

In [134]:
p = LpProblem('payload', LpMinimize)

In [135]:
# decision variables
variables = []
for i in range(1,16):
    for j in 'ABCD':
        x = LpVariable('x{}{}'.format(i,j), 0, 1, LpBinary)
        variables.append(x)
len(variables)

60

In [136]:
# two dummy variables for the objective function
t1 = LpVariable('t1', 0, 1000, LpContinuous)
t2 = LpVariable('t2', 0, 1000, LpContinuous)

In [137]:
# add the objective to the problem
p += t1+t2

In [138]:
# build the weight vector
w = np.array([70, 90, 100, 110, 120, 130, 150, 180, 210, 220, 250, 280, 340, 350, 400])

In [139]:
# add the constraints on t1 and t2 to replace the absolute values
portA = []
for j in range(0,60,4):
    portA.append(variables[j])
    
portB = []
for j in range(1,60,4):
    portB.append(variables[j])
    
portC = []
for j in range(2,60,4):
    portC.append(variables[j])
    
portD = []
for j in range(3,60,4):
    portD.append(variables[j])

In [140]:
# add the dummy constraints
p+= t1==(lpDot(portA,w)-lpDot(portD,w))
#p+= t1>= -(lpDot(portA,w)-lpDot(portD,w))
p+= t2==(lpDot(portB,w)-lpDot(portC,w))
#p+= t2>= -(lpDot(portB,w)-lpDot(portC,w))

In [141]:
# weight limits in each cargo port
p+=(lpDot(portA,w)<=1000)
p+=(lpDot(portB,w)<=1000)
p+=(lpDot(portC,w)<=1000)
p+=(lpDot(portD,w)<=1000)

In [142]:
# the requirement that we have at least three packages in each
p+= lpSum(portA)>=3
p+= lpSum(portB)>=3
p+= lpSum(portC)>=3
p+= lpSum(portD)>=3

In [143]:
# we also need that each package goes in only one port
for j in range(0,60,4):
    p+= variables[j]+variables[j+1]+variables[j+2]+variables[j+3] == 1

In [144]:
status = p.solve()

In [145]:
print LpStatus[status]

Optimal


In [146]:
for j in variables:
    if j.value():
        print j

x1D
x2D
x3D
x4A
x5C
x6B
x7C
x8A
x9B
x10B
x11A
x12D
x13C
x14C
x15B


In [147]:
# cool! we've minimized the function we were hoping for
print value(p.objective)

0.0


In [148]:
# now to answer the question of the weights of each storage bay
weightA = 0
for j in range(0,len(portA)):
     weightA = weightA + (portA[j].value())*w[j]

weightB = 0
for j in range(0,len(portB)):
     weightB = weightB + (portB[j].value())*w[j]
        
weightC = 0
for j in range(0,len(portC)):
     weightC = weightC + (portC[j].value())*w[j]
        
weightD = 0
for j in range(0,len(portD)):
     weightD = weightD + (portD[j].value())*w[j]

In [149]:
print weightA
print weightB
print weightC
print weightD

540.0
960.0
960.0
540.0


In [150]:
print lpDot(portA,w).value()

540.0


In [151]:
print lpDot(portB,w).value()

960.0


In [152]:
print lpDot(portC,w).value()

960.0


In [153]:
t1.value()

0.0